# Calculate SAM index and plot results
- opens all CMIP6 historical model run files with data on surface temperature tas
- open Abram and Marshall data
- calculates the SAM index for the data to test the SAM state in the months before and after Krakatoa 1883


In [1]:
import xarray as xr, matplotlib.pyplot as plt
from importlib import reload # need to use this if I edit a function file
import os
import numpy as np
import pandas as pd
import cartopy.crs as ccrs # to add in continents and change map projections 
from matplotlib.colors import LinearSegmentedColormap # to change colour bar????
import dask.diagnostics # dask allows you to check how long something is taking to load
import climtas # needed to count event statistics with a specified duration

In [2]:
# import custom functions
import sys 
sys.path.append('/home/563/kb6999/Functions') # use this if the function file is in a different directory to the notebook

import frequently_used_functions as func
import plotting_functions as fplot
import model_functions as funcM
import reanalysis_functions as funcR

In [3]:
!ls /g/data/lp01/CMIP6/CMIP/CAS/CAS-ESM2-0/historical/r1i1p1f1/Amon/psl/gr1.5/v20200302


psl_Amon_CAS-ESM2-0_historical_r1i1p1f1_gr1.5_185001-201412.nc


In [4]:
# store each section of the directory as a string
institution_dir = '/g/data/lp01/CMIP6/CMIP/'
tas_dir = '/historical/r1i1p1f1/Amon/tas/gr1.5/'
pr_dir = '/historical/r1i1p1f1/Amon/pr/gr1.5/'
psl_dir = '/historical/r1i1p1f1/Amon/psl/gr1.5/'
# ts_dir = '/historical/r1i1p1f1/Amon/ts/gr1.5/'
print(institution_dir, tas_dir, pr_dir, psl_dir)

/g/data/lp01/CMIP6/CMIP/ /historical/r1i1p1f1/Amon/tas/gr1.5/ /historical/r1i1p1f1/Amon/pr/gr1.5/ /historical/r1i1p1f1/Amon/psl/gr1.5/


## Read in model data

In [43]:
models_psl = funcM.read_models(institution_dir, psl_dir, '1850-01','2015-01').chunk(chunks={'time': 1000})

52 model paths found and loaded into the dictionary "models"
Path for MCM-UA-1-0 does not exist
51 models have been successfully loaded into an xarray


In [44]:
# 43 models i'm using in my project, common to both tas and pr
models_43 = ['ACCESS-CM2','ACCESS-ESM1-5','AWI-CM-1-1-MR','AWI-ESM-1-1-LR','BCC-CSM2-MR','BCC-ESM1',
                             'CAMS-CSM1-0', 'CAS-ESM2-0', 'CESM2', 'CESM2-FV2', 'CESM2-WACCM', 'CESM2-WACCM-FV2', 
                             'CMCC-CM2-HR4', 'CMCC-CM2-SR5' ,'CanESM5', 'E3SM-1-1' ,'E3SM-1-1-ECA' ,'EC-Earth3', 
                             'EC-Earth3-AerChem' , 'EC-Earth3-Veg-LR', 'FGOALS-f3-L' ,'FGOALS-g3', 'FIO-ESM-2-0', 
                             'GFDL-CM4' ,'GFDL-ESM4', 'GISS-E2-1-G' ,'GISS-E2-1-G-CC', 'GISS-E2-1-H' ,'IITM-ESM', 
                             'INM-CM4-8', 'INM-CM5-0' ,'IPSL-CM6A-LR', 'KACE-1-0-G', 'MIROC6' ,'MPI-ESM-1-2-HAM',
                             'MPI-ESM1-2-LR' ,'MRI-ESM2-0', 'NESM3', 'NorCPM1' ,'NorESM2-LM', 'NorESM2-MM',
                             'SAM0-UNICON', 'TaiESM1']

In [45]:
# select out the 43 models i'm using for tas and pr 
# models_tas = models_tas.sel(model= models_43)
# models_pr = models_pr.sel(model= models_43)
models_psl = models_psl.sel(model=models_43)
models_psl

,Array,Chunk
Bytes,9.13 GiB,109.86 MiB
Shape,"(43, 1980, 120, 240)","(1, 1000, 120, 240)"
Count,25586 Tasks,86 Chunks
Type,float32,numpy.ndarray


In [46]:
# # rechunk data 
# models_psl = models_psl.chunk(chunks={'time': 200})
# models_psl

In [47]:
# sort models alphabetically and prints final model list
models_psl = models_psl.sortby('model')
print('The following', len(models_psl.model.data), 'models will be used in all subsequent calculations: \n')
print(models_psl.model.data)

The following 43 models will be used in all subsequent calculations: 

['ACCESS-CM2' 'ACCESS-ESM1-5' 'AWI-CM-1-1-MR' 'AWI-ESM-1-1-LR'
 'BCC-CSM2-MR' 'BCC-ESM1' 'CAMS-CSM1-0' 'CAS-ESM2-0' 'CESM2' 'CESM2-FV2'
 'CESM2-WACCM' 'CESM2-WACCM-FV2' 'CMCC-CM2-HR4' 'CMCC-CM2-SR5' 'CanESM5'
 'E3SM-1-1' 'E3SM-1-1-ECA' 'EC-Earth3' 'EC-Earth3-AerChem'
 'EC-Earth3-Veg-LR' 'FGOALS-f3-L' 'FGOALS-g3' 'FIO-ESM-2-0' 'GFDL-CM4'
 'GFDL-ESM4' 'GISS-E2-1-G' 'GISS-E2-1-G-CC' 'GISS-E2-1-H' 'IITM-ESM'
 'INM-CM4-8' 'INM-CM5-0' 'IPSL-CM6A-LR' 'KACE-1-0-G' 'MIROC6'
 'MPI-ESM-1-2-HAM' 'MPI-ESM1-2-LR' 'MRI-ESM2-0' 'NESM3' 'NorCPM1'
 'NorESM2-LM' 'NorESM2-MM' 'SAM0-UNICON' 'TaiESM1']


## SAM index

In [48]:
# calculate the zonal mean mslp at 40 and 65S (to be used in calculating SAM)
P40S = models_psl.sel(lat=-39.75).mean(dim='lon')
P65S = models_psl.sel(lat=-65.25).mean(dim='lon')
P65S

<xarray.Dataset>
Dimensions:  (model: 43, time: 1980)
Coordinates:
    lat      float64 -65.25
  * time     (time) datetime64[ns] 1850-01-31 1850-02-28 ... 2014-12-31
  * model    (model) <U17 'ACCESS-CM2' 'ACCESS-ESM1-5' ... 'TaiESM1'
Data variables:
    psl      (model, time) float32 dask.array<chunksize=(1, 1000), meta=np.ndarray>

In [49]:
# set the dates for the normalisation period
start_date = '1971'
end_date = '2000'

In [50]:
# calculate the anomaly for the normalised zonal mean, x, (x-mean)/std
anom40 = func.monthly_anomaly(P40S, start_date, end_date)
anom65 = func.monthly_anomaly(P65S, start_date, end_date)
anom65

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.04/lib/python3.8/site-packages/xarray/core/indexing.py:1379: PerformanceWarning: Slicing with an out-of-order index is generating 83 times more chunks
  return self.array[key]
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.04/lib/python3.8/site-packages/xarray/core/indexing.py:1379: PerformanceWarning: Slicing with an out-of-order index is generating 83 times more chunks
  return self.array[key]


<xarray.Dataset>
Dimensions:  (model: 43, time: 1980)
Coordinates:
    lat      float64 -65.25
  * time     (time) datetime64[ns] 1850-01-31 1850-02-28 ... 2014-12-31
  * model    (model) <U17 'ACCESS-CM2' 'ACCESS-ESM1-5' ... 'TaiESM1'
    month    (time) int64 1 2 3 4 5 6 7 8 9 10 11 ... 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    psl      (model, time) float32 dask.array<chunksize=(1, 1), meta=np.ndarray>

In [51]:
# calculate the std for the normalised zonal mean, x, (x-mean)/std
std40 = anom40.sel(time=slice(start_date, end_date)).std(dim='time')
std65 = anom65.sel(time=slice(start_date, end_date)).std(dim='time')
std40

<xarray.Dataset>
Dimensions:  (model: 43)
Coordinates:
    lat      float64 -39.75
  * model    (model) <U17 'ACCESS-CM2' 'ACCESS-ESM1-5' ... 'TaiESM1'
Data variables:
    psl      (model) float32 dask.array<chunksize=(1,), meta=np.ndarray>

In [52]:
# normalise the pressure values by minusing the mean (ie calculating anomaly) and dividing by 30-year average
P40S_n = anom40/std40
P65S_n = anom65/std65

In [53]:
# calculate the SAM index by finding the normalised pressure difference between the two latitude bands
SAM = P40S_n - P65S_n
SAM

<xarray.Dataset>
Dimensions:  (model: 43, time: 1980)
Coordinates:
  * time     (time) datetime64[ns] 1850-01-31 1850-02-28 ... 2014-12-31
  * model    (model) <U17 'ACCESS-CM2' 'ACCESS-ESM1-5' ... 'TaiESM1'
    month    (time) int64 1 2 3 4 5 6 7 8 9 10 11 ... 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    psl      (model, time) float32 dask.array<chunksize=(1, 1), meta=np.ndarray>

### save data to netcdf - kernel keeps dying here!!

In [54]:
path_models = '/g/data/w48/kb6999/Models/'
path_r = '/g/data/w48/kb6999/'

In [ ]:
with dask.diagnostics.ProgressBar():
    SAM.psl.to_netcdf(f'{path_models}M_SAM_index.nc')

[                                        ] | 0% Completed | 35.3s

In [19]:
# read in Abram data SAM index
SAM_abram = xr.open_dataset(f'{path_r}SAM_data_abram_1850_2007.nc')
SAM_abram

<xarray.Dataset>
Dimensions:                    (time: 158)
Coordinates:
  * time                       (time) datetime64[ns] 1850-01-01 ... 2007-01-01
Data variables:
    SAM_Index                  (time) float64 ...
    SAM_Index_err              (time) float64 ...
    RE                         (time) float64 ...
    RE_crit                    (time) float64 ...
    SAM_Index_70yloess         (time) float64 ...
    SAM_Index_+error_70yloess  (time) float64 ...
    SAM_Index_-error_70yloess  (time) float64 ...
    Marshall_SAM_Index         (time) float64 ...

# figures

In [15]:
# set the default font size
SMALL_SIZE = 10
MEDIUM_SIZE = 12
BIGGER_SIZE = 14

plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [16]:
e_date = ['1883-08-27']

In [ ]:
fig, axes = plt.subplots(3, 1, sharex=True, figsize=(12,12))

start_date, end_date = '1957','2007'
ds = SAM.psl.sel(time=slice(start_date, end_date)).resample(time='YS').mean(dim='time')
fplot.SAM_plot(ds, ax = axes[0])

ds1 = SAM_abram.SAM_Index.sel(time=slice(start_date, end_date))
fplot.SAM_plot(ds1, ax = axes[1])

ds2 = SAM_abram.Marshall_SAM_Index.sel(time=slice(start_date, end_date))
fplot.SAM_plot(ds2, ax = axes[2])

#set the frequency of the xticks 
years = pd.date_range(pd.to_datetime(start_date)-2* pd.offsets.YearBegin(), 
                      pd.to_datetime(end_date)+3* pd.offsets.YearBegin(), freq='5YS')
axes[0].set_xticks(years.values)
axes[0].set_xticklabels(years.year) # .year shows only the year (not month)

# set axis titles
axes[0].set(title = 'Twentieth Century Reanalysis')
axes[1].set(title = 'Abram (2010)')
axes[2].set(title = 'Marshall')

plt.savefig('M_timeseries_year_SAM_compare.png', dpi=300, bbox_inches='tight')